In [1]:
def get_macro_and_micro(dict, file,nam):
    type= nam.split("_")[0]
    name = (nam.split("_")[-1].split(".json")[0])
    #print(name)
    for line in file :
        
        if "micro" in line :
                               
            dict["micro"]["precision"][name][type]=line.split("        ")[2]
            dict["micro"]["recall"][name][type]=line.split("        ")[3]
            dict["micro"]["f1-score"][name][type]=line.split("        ")[4]
        if "macro" in line :
            
            dict["macro"]["precision"][name][type]=line.split("        ")[2] 
            dict["macro"]["recall"][name][type]=line.split("        ")[3]
            dict["macro"]["f1-score"][name][type]=line.split("        ")[4]
            #print("**macro",dict)
            return dict 
        
def get_result_by_threshold(threshold):
    result = "CONLL04/result_"+str(threshold)+".txt"
    
    f = open(result, "r")   
    all=[]
    dict={}
    dict["NER"]={}
    dict["RE"]={}
    dict["NER-RE"]={}
    for dicte in [ dict["NER-RE"],  dict["NER"], dict["RE"]] :
        dicte["micro"]={}
        dicte["micro"]["precision"]={}
        dicte["micro"]["recall"]={}
        dicte["micro"]["f1-score"]={}
        dicte["macro"]={}
        dicte["macro"]["precision"]={}
        dicte["macro"]["recall"]={}
        dicte["macro"]["f1-score"]={}
        for dicte in [dicte["macro"]["f1-score"],dicte["macro"]["precision"],dicte["macro"]["recall"],
                      dicte["micro"]["precision"],dicte["micro"]["recall"],dicte["micro"]["f1-score"]] : 
            for id in range(1,21,1):
                dicte[str(id)]={}
        
    for line in f :
        if "###"  in str(line):
            name =(line.split("#")[-1]).replace(" ","").replace("\n","")   
            
        if "Entities (named entity recognition (NER))" in str(line):
            
            dict["NER"]=get_macro_and_micro(dict["NER"],f,name)
        if "Without named entity classification (NEC)" in str(line):
            
            dict["RE"] =get_macro_and_micro(dict["RE"],f,name)
        if "With named entity classification (NEC)" in str(line):
            
            dict["NER-RE"] =get_macro_and_micro(dict["NER-RE"],f,name)
            
    #print(all) 
    f.close()
    return dict

thresholds=[0.4]
result={}
for threshold in thresholds :
    result[str(threshold)]=get_result_by_threshold(threshold)

print(result)

{'0.4': {'NER': {'micro': {'precision': {'1': {'predictions': '32.00', 'neurASP': '32.00', 'neurASP-FE': '32.00', 'neurASP-FE-SUP': '32.00', 'neurASP-SUP': '32.00'}, '2': {'predictions': '59.43', 'neurASP': '59.51', 'neurASP-FE': '59.43', 'neurASP-FE-SUP': '59.43', 'neurASP-SUP': '59.60'}, '3': {}, '4': {}, '5': {}, '6': {}, '7': {}, '8': {}, '9': {}, '10': {'predictions': '85.27', 'neurASP': '85.01', 'neurASP-FE': '85.27', 'neurASP-FE-SUP': '85.27', 'neurASP-SUP': '85.10'}, '11': {'predictions': '86.34', 'neurASP': '86.34', 'neurASP-FE': '86.34', 'neurASP-FE-SUP': '86.34', 'neurASP-SUP': '86.34'}, '12': {'predictions': '87.31', 'neurASP': '87.31', 'neurASP-FE': '87.31', 'neurASP-FE-SUP': '87.31', 'neurASP-SUP': '87.31'}, '13': {'predictions': '86.64', 'neurASP': '86.64', 'neurASP-FE': '86.64', 'neurASP-FE-SUP': '86.64', 'neurASP-SUP': '86.64'}, '14': {'predictions': '87.81', 'neurASP': '87.72', 'neurASP-FE': '87.81', 'neurASP-FE-SUP': '87.81', 'neurASP-SUP': '87.81'}, '15': {'predicti

In [12]:
# display it with tensorboard
from torch.utils.tensorboard import SummaryWriter
import numpy as np
database="CONLL04"
metrics=["precision","recall","f1-score"]
neurasp={}
spert ={}
writer = SummaryWriter(database+"/Threshold")

for threshold,values_parts in result.items():
    #0.1
        for key_type,type_evaluation in values_parts.items() :
            print(key_type)
            #NER
            for key_eval, evaluation in type_evaluation.items() :
                #micro
                
                for metric, values in evaluation.items() :
                        #precision
                    if key_eval =="micro" and (key_type=="NER-RE" or key_type=="RE" ): 
                        print(values)
                        
                        for epoch, value in values.items():
                            if value!={} :
                            ####################après enlever pour qu'on ait aussi le micro 
                                print(epoch, " ", threshold)
                                writer.add_scalars("Test-"+database+" "+str("runs/"+threshold+"/"+key_type+"/"+key_eval+"/"+metric) ,
                                                   {"SPERT": float(value["predictions"]), 
                                                    "SPERT-Neur": float(value["neurASP"]), "SPERT-Neur-FE": float(value["neurASP-FE"]),
                                                    "SPERT-Neur-FE-SUP": float(value["neurASP-FE-SUP"]),
                                                   "SPERT-Neur-SUP": float(value["neurASP-SUP"])}, int(epoch))
                                #ceci c'est en function du threshold"""
                                if database =="scierc":
                                    writer.add_scalars("Test-epoch"+threshold+" "+database+" "+str(key_type+"/"+key_eval+"/"+metric) ,
                                                   {"SPERT": float(value["predictions"]), 
                                                    "SPERT-Neur": float(value["neurASP"]), "SPERT-Neur-FE": float(value["neurASP-FE"]),
                                                    "SPERT-Neur-FE-SUP": float(value["neurASP-FE-SUP"]),
                                                   "SPERT-Neur-SUP": float(value["neurASP-SUP"])}, float(epoch))
                                """ else :
                                    writer.add_scalars("Test-epoch"+threshold+" "+database+" "+str(key_type+"/"+key_eval+"/"+metric) ,
                                                   {"SPERT": float(value["predictions"]), 
                                                    "SPERT-Neur": float(value["neurASP"]), "SPERT-Neur-FE": float(value["neurASP-FE"]),
                                                    "SPERT-Neur-FE-SUP": float(value["neurASP-FE-SUP"])}, float(epoch)*100)"""
writer.close()

NER
RE
{'1': {'predictions': ' 2.06', 'neurASP': ' 2.06', 'neurASP-FE': ' 2.06', 'neurASP-FE-SUP': ' 2.07', 'neurASP-SUP': ' 2.75'}, '2': {'predictions': '25.40', 'neurASP': '25.40', 'neurASP-FE': '25.40', 'neurASP-FE-SUP': '25.06', 'neurASP-SUP': '25.66'}, '3': {}, '4': {}, '5': {}, '6': {}, '7': {}, '8': {}, '9': {}, '10': {'predictions': '69.62', 'neurASP': '69.11', 'neurASP-FE': '69.62', 'neurASP-FE-SUP': '70.93', 'neurASP-SUP': '71.05'}, '11': {'predictions': '67.23', 'neurASP': '67.47', 'neurASP-FE': '67.47', 'neurASP-FE-SUP': '68.09', 'neurASP-SUP': '68.54'}, '12': {'predictions': '70.63', 'neurASP': '70.63', 'neurASP-FE': '70.63', 'neurASP-FE-SUP': '70.50', 'neurASP-SUP': '71.77'}, '13': {'predictions': '68.78', 'neurASP': '68.54', 'neurASP-FE': '68.54', 'neurASP-FE-SUP': '69.02', 'neurASP-SUP': '69.39'}, '14': {'predictions': '66.43', 'neurASP': '66.19', 'neurASP-FE': '66.43', 'neurASP-FE-SUP': '67.00', 'neurASP-SUP': '67.75'}, '15': {'predictions': '69.88', 'neurASP': '69.38'

20   0.4
{'1': {'predictions': ' 1.50', 'neurASP': ' 1.50', 'neurASP-FE': ' 1.50', 'neurASP-FE-SUP': ' 1.51', 'neurASP-SUP': ' 1.66'}, '2': {'predictions': '25.73', 'neurASP': '25.73', 'neurASP-FE': '25.73', 'neurASP-FE-SUP': '25.21', 'neurASP-SUP': '25.51'}, '3': {}, '4': {}, '5': {}, '6': {}, '7': {}, '8': {}, '9': {}, '10': {'predictions': '67.07', 'neurASP': '66.83', 'neurASP-FE': '67.32', 'neurASP-FE-SUP': '66.75', 'neurASP-SUP': '66.67'}, '11': {'predictions': '66.43', 'neurASP': '66.91', 'neurASP-FE': '66.91', 'neurASP-FE-SUP': '66.10', 'neurASP-SUP': '65.93'}, '12': {'predictions': '68.05', 'neurASP': '68.30', 'neurASP-FE': '68.30', 'neurASP-FE-SUP': '67.08', 'neurASP-SUP': '67.92'}, '13': {'predictions': '67.55', 'neurASP': '67.55', 'neurASP-FE': '67.55', 'neurASP-FE-SUP': '66.91', 'neurASP-SUP': '66.83'}, '14': {'predictions': '66.27', 'neurASP': '66.27', 'neurASP-FE': '66.51', 'neurASP-FE-SUP': '65.70', 'neurASP-SUP': '65.94'}, '15': {'predictions': '68.20', 'neurASP': '67.9

TypeError: add_scalars() got an unexpected keyword argument 'colors'